In [1]:
import nltk
import re

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
paragraph = """
Chief Bola Ahmed Adekunle Tinubu GCFR (born 29 March 1952) is a Nigerian politician who is the 16th and current president of Nigeria.[1] He was the governor of Lagos State from 1999 to 2007; and senator for Lagos West in the Third Republic.

Tinubu spent his early life in southwestern Nigeria and later moved to the United States where he studied accounting at Chicago State University. He returned to Nigeria in the early 1990s and was employed by Mobil Nigeria as an accountant, before entering politics as a Lagos West senatorial candidate in 1992 under the banner of the Social Democratic Party. After dictator Sani Abacha dissolved the Senate in 1993, Tinubu became an activist campaigning for the return of democracy as a part of the National Democratic Coalition movement.

In the first post-transition Lagos State gubernatorial election, Tinubu won by a wide margin as a member of the Alliance for Democracy. Four years later, he won re-election to a second term. After leaving office in 2007, he played a key role in the formation of the All Progressives Congress in 2013. In 2023, he was elected president of Nigeria.
"""

In [4]:
# tokenization
nltk.download('stopwords')
nltk.download('punkt')

sentences = nltk.sent_tokenize(paragraph)

[nltk_data] Downloading package stopwords to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stemmer = PorterStemmer()
lemmatize = WordNetLemmatizer()


In [5]:
corpus = []
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review=review.lower()
    corpus.append(review)

In [ ]:
# to apply stemming
for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(stemmer.stem(word))

In [6]:
stopwords.words('english')
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# paragraph = []

# for i in range(len(sentences)):
#     review = re.sub('[^a-zA-Z]', ' ', sentences[i])
#     review = review.lower()
#     review = review.split()
#     review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
#     review = ' '.join(review) 
#     paragraph.append(review)

In [ ]:
# to apply lemmatizer

for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(lemmatize.lemmatize(word))

In [11]:
# bag of words

cv = CountVectorizer(binary=True, ngram_range=(3,3))

In [ ]:
X = cv.fit_transform(corpus)
cv.vocabulary_
sorted(cv.vocabulary_.items())


In [14]:
X[0].toarray()

array([[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0]], dtype=int64)

In [ ]:
corpus

In [36]:
## TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(ngram_range=(1,1), max_features=10)
X = tv.fit_transform(corpus)

In [37]:
X[0].toarray()

array([[0.49728801, 0.        , 0.        , 0.        , 0.        ,
        0.49728801, 0.35924498, 0.35924498, 0.49728801, 0.        ]])

In [47]:
import pandas as pd
spam = pd.read_csv('spam_ham.csv', sep='\t', names=['labels', 'message'])

In [52]:
corpus_stem = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus_stem.append(review)

In [54]:
# bag of words
cv = CountVectorizer(max_features=2500,binary=True)
X = cv.fit_transform(corpus_stem).toarray()

In [62]:
y = pd.get_dummies(spam['labels'])
y = y.iloc[:, 1]

In [64]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [91]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train, y_train)

In [72]:
# prediction
y_pred = mnb.predict(X_test)

In [77]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [75]:
accuracy_score(y_pred, y_test)

0.9865470852017937

In [80]:
classification_report(y_pred, y_test)

'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99       960\n           1       0.94      0.97      0.95       155\n\n    accuracy                           0.99      1115\n   macro avg       0.97      0.98      0.97      1115\nweighted avg       0.99      0.99      0.99      1115\n'

In [76]:
confusion_matrix(y_pred, y_test)

array([[950,  10],
       [  5, 150]], dtype=int64)

In [81]:
#  Creating TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
tv = TfidfVectorizer(max_features=2500, ngram_range=(1,2))
X = tv.fit_transform(corpus_stem).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [92]:
mnb = MultinomialNB().fit(X_train, y_train)

In [93]:
y_pred = mnb.predict(X_test)
accuracy_score(y_pred, y_test)

0.9811659192825112

In [50]:
corpus = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus.append(review)